## load logistic and lorenz logs

In [38]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib notebook

In [39]:
# fig = plt.figure(figsize=(20,3.5))
# ax = plt.gca()

def plot_ts(pred):
    y_true, y_pred =pred
    ax = fig.add_subplot(1, 4, i+1)
    ax.plot(y_true[:100,seq_id],'g')
    ax.set_ylim([-20,20])
    ax = fig.add_subplot(2, 4, i+1)
    ax.plot(y_pred[:100,seq_id], 'r')
    ax.set_ylim([-20,20])
    ax.set_title(model)

### load logs for lorenz and print test error

In [40]:
time_stamp ="06-16-17-00-29-37"
exp='traffic'
models =["basic_rnn","basic_lstm","matrix_rnn","tensor_rnn"]
hz_list= [32, 64, 128, 256]
lr_list =[1,2,3,4]


import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt



def read_config(fstream):
    d = {}
    for line in fstream:
        line.rstrip()
        tokens = line.split('\t')
        for token in tokens:
            try:
                key, val = token.split(":")
                d[key] = float(val)                
            except ValueError:
                'error prop'
    return d

def load_log(hz, lr, model):
    data_path = "/var/tmp/tensorRNN/log/"+exp+"/"+time_stamp+"/hz_"+str(hz)+"/"+"lr_1e-"+str(lr)+"/"+model+"/"
    f_pred= "predict.npy"
    f_err = "config_error.out"
    pred =  np.load(data_path+f_pred )
    with open(data_path+"/"+f_err,'r') as fstream:
        config = read_config(fstream)
    return config, pred

errs = np.ndarray((len(hz_list), len(lr_list), len(models)))
i=0
for hz in hz_list:
    j=0
    for lr in lr_list:
        k=0
        for model in models:
            print('load model {:s}, hidden size {:d}, learning rate 1e-{:d}'.format(model, hz, lr))
            config, pred = load_log(hz, lr, model)
            print('train error:',config['train_error'],'valid error:',config['valid_error'],'test err:',config['test_error'])
            errs[i,j,k] =  config['test_error']
            k +=1
        j+=1

    i+=1

load model basic_rnn, hidden size 32, learning rate 1e-1
train error: 0.309181871253 valid error: 0.367979316189 test err: 0.260771803993
load model basic_lstm, hidden size 32, learning rate 1e-1
train error: 0.10818775754 valid error: 0.120031006065 test err: 0.110499031712
load model matrix_rnn, hidden size 32, learning rate 1e-1
train error: 0.130272894586 valid error: 0.132599037816 test err: 0.128421029247
load model tensor_rnn, hidden size 32, learning rate 1e-1
train error: 0.214370016502 valid error: 0.175594666457 test err: 0.179219113541
load model basic_rnn, hidden size 32, learning rate 1e-2
train error: 0.160391982428 valid error: 0.138496734182 test err: 0.145000507718
load model basic_lstm, hidden size 32, learning rate 1e-2
train error: 0.0840866621709 valid error: 0.0929795292669 test err: 0.0913323272641
load model matrix_rnn, hidden size 32, learning rate 1e-2
train error: 0.092550626726 valid error: 0.0942106248607 test err: 0.0906818752377
load model tensor_rnn, hi

In [43]:
np.set_printoptions(precision=3)
for k in range(len(models)):
    print(np.min(errs[:,:,k])) #row min--hidden size

0.112667154587
0.0859811125377
0.0901672604441
0.0891145993806
